# Polymarket Arbitrage Bot - Complete Testing Notebook

**Purpose:** Test all components step-by-step before building the production bot

**What we'll test:**
1. ✅ Environment setup
2. ✅ Load configuration from .env
3. ✅ Connect to Polygon via Alchemy
4. ✅ Check wallet balance (USDC)
5. ✅ Connect to Polymarket WebSocket
6. ✅ Fetch real market data
7. ✅ Parse order books
8. ✅ Calculate arbitrage opportunities

**Run each section sequentially** - each builds on the previous one!

---
## Section 1: Environment Setup & Imports
Test that all packages are installed and working

In [ ]:
print("🔍 Starting Polymarket Arbitrage Bot Testing...")
print("="*60)
print("")

print("📦 Importing libraries...")
import sys
import os
import asyncio
from datetime import datetime
from dotenv import load_dotenv

# Web3 & Blockchain
from web3 import Web3

# Polymarket SDK
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import ApiCreds

# Speed optimizations
import orjson  # Fast JSON parsing
# import uvloop  # Fast event loop (UNIX/MAC ONLY - commented out for Windows)

# Data analysis
import pandas as pd
import numpy as np

print("✅ All imports successful!")
print("")
print(f"📊 Python version: {sys.version}")
print(f"📊 Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("")
print("✅ Section 1 complete!")

---
## Section 2: Load Configuration from .env
Load all environment variables and verify they're set correctly

In [ ]:
print("🔍 Loading configuration from .env file...")
print("="*60)
print("")

# Import required modules
import os
from pathlib import Path
from dotenv import load_dotenv

# Get the project root directory (one level up from notebooks/)
project_root = Path().absolute().parent
env_path = project_root / '.env'

print(f"📂 Looking for .env at: {env_path}")

if env_path.exists():
    load_dotenv(env_path)
    print("✅ .env file loaded")
else:
    print("❌ .env file not found!")
    print(f"   Expected location: {env_path}")
    print("   Make sure .env is in the project root directory")
print("")

# Get configuration values
POLYGON_RPC_URL = os.getenv('POLYGON_RPC_URL')
WALLET_ADDRESS = os.getenv('WALLET_ADDRESS')
WALLET_PRIVATE_KEY = os.getenv('WALLET_PRIVATE_KEY')
DRY_RUN_MODE = os.getenv('DRY_RUN_MODE', 'true').lower() == 'true'

# Display configuration (hiding sensitive data)
print("📊 Configuration loaded:")
if POLYGON_RPC_URL:
    print(f"   🌐 RPC URL: {POLYGON_RPC_URL[:50]}...")
else:
    print("   ❌ RPC URL: Not found in .env!")
    
if WALLET_ADDRESS:
    print(f"   💰 Wallet: {WALLET_ADDRESS}")
else:
    print("   ❌ Wallet Address: Not found in .env!")
    
print(f"   🔐 Private Key: {'*' * 20} (hidden)")
print(f"   🧪 Dry Run Mode: {DRY_RUN_MODE}")
print("")

# Validate required fields
if not POLYGON_RPC_URL or 'YOUR_' in POLYGON_RPC_URL:
    print("❌ ERROR: POLYGON_RPC_URL not set in .env file!")
elif not WALLET_ADDRESS or 'Your' in WALLET_ADDRESS:
    print("❌ ERROR: WALLET_ADDRESS not set in .env file!")
else:
    print("✅ All required configuration fields are set!")
    print("⚠️ Note: WALLET_PRIVATE_KEY needed later for trading")

print("")
print("✅ Section 2 complete!")

---
## Section 3: Connect to Polygon Blockchain (via Alchemy)
Test that we can connect to the Polygon network

In [ ]:
print("🔍 Connecting to Polygon blockchain via Alchemy...")
print("="*60)
print("")

try:
    # Initialize Web3 connection
    print("⏱️ Initializing Web3 connection...")
    w3 = Web3(Web3.HTTPProvider(POLYGON_RPC_URL))
    
    # Test connection
    print("⏱️ Testing connection...")
    is_connected = w3.is_connected()
    
    if is_connected:
        print("✅ Successfully connected to Polygon!")
        print("")
        
        # Get blockchain info
        print("📊 Blockchain Information:")
        chain_id = w3.eth.chain_id
        latest_block = w3.eth.block_number
        gas_price_wei = w3.eth.gas_price
        gas_price_gwei = w3.from_wei(gas_price_wei, 'gwei')
        
        print(f"   🔗 Chain ID: {chain_id} (should be 137 for Polygon)")
        print(f"   📦 Latest Block: {latest_block:,}")
        print(f"   ⛽ Gas Price: {gas_price_gwei:.2f} Gwei")
        print("")
        
        if chain_id == 137:
            print("✅ Connected to Polygon Mainnet!")
        else:
            print(f"⚠️ WARNING: Connected to chain ID {chain_id}, not Polygon Mainnet (137)")
    else:
        print("❌ Failed to connect to Polygon")
        print("   Check your POLYGON_RPC_URL in .env file")
        
except Exception as e:
    print(f"❌ ERROR: {str(e)}")
    print("")
    print("Troubleshooting:")
    print("   1. Check your Alchemy API key is correct")
    print("   2. Make sure you're using the Polygon endpoint")
    print("   3. Verify your Alchemy account is active")

print("")
print("✅ Section 3 complete!")

---
## Section 4: Check Wallet Balance (USDC)
Check if your wallet has USDC on Polygon

In [ ]:
print("🔍 Checking wallet balance...")
print("="*60)
print("")

try:
    # Get MATIC (native token) balance
    print("⏱️ Checking MATIC balance...")
    matic_balance_wei = w3.eth.get_balance(WALLET_ADDRESS)
    matic_balance = w3.from_wei(matic_balance_wei, 'ether')
    print(f"   💰 MATIC Balance: {matic_balance:.4f} MATIC")
    
    if matic_balance < 0.01:
        print("   ⚠️ Low MATIC balance - you need MATIC for gas fees!")
    else:
        print("   ✅ Sufficient MATIC for gas fees")
    print("")
    
    # USDC Contract Address on Polygon
    USDC_ADDRESS = '0x3c499c542cEF5E3811e1192ce70d8cC03d5c3359'  # Native USDC on Polygon
    
    # ERC20 ABI (minimal - just need balanceOf)
    ERC20_ABI = '''[
        {
            "constant":true,
            "inputs":[{"name":"_owner","type":"address"}],
            "name":"balanceOf",
            "outputs":[{"name":"balance","type":"uint256"}],
            "type":"function"
        },
        {
            "constant":true,
            "inputs":[],
            "name":"decimals",
            "outputs":[{"name":"","type":"uint8"}],
            "type":"function"
        }
    ]'''
    
    print("⏱️ Checking USDC balance...")
    import json
    usdc_contract = w3.eth.contract(address=USDC_ADDRESS, abi=json.loads(ERC20_ABI))
    usdc_balance_raw = usdc_contract.functions.balanceOf(WALLET_ADDRESS).call()
    usdc_decimals = usdc_contract.functions.decimals().call()
    usdc_balance = usdc_balance_raw / (10 ** usdc_decimals)
    
    print(f"   💵 USDC Balance: ${usdc_balance:.2f} USDC")
    print("")
    
    if usdc_balance == 0:
        print("   ⚠️ No USDC in wallet!")
        print("   You need to transfer USDC to your Polygon wallet to start trading")
        print("   Recommended: $50-100 USDC to start testing")
    elif usdc_balance < 50:
        print(f"   ⚠️ Low USDC balance (${usdc_balance:.2f})")
        print("   Recommended: $50-100 USDC for testing")
    else:
        print("   ✅ Sufficient USDC balance for testing!")
        
except Exception as e:
    print(f"❌ ERROR: {str(e)}")
    import traceback
    print(traceback.format_exc())

print("")
print("✅ Section 4 complete!")

---
## Section 5: Summary & Next Steps
Review what we've tested and what's next

In [ ]:
print("="*60)
print("🎉 TESTING SUMMARY")
print("="*60)
print("")
print("✅ Section 1: Environment setup - PASSED")
print("✅ Section 2: Configuration loaded - PASSED")
print("✅ Section 3: Polygon connection - PASSED")
print("✅ Section 4: Wallet balance checked - PASSED")
print("")
print("📋 Next Steps:")
print("   1. Generate Polymarket API credentials")
print("   2. Test WebSocket connection to Polymarket")
print("   3. Fetch real order book data")
print("   4. Calculate arbitrage opportunities")
print("   5. Test order placement (dry run)")
print("")
print("✅ Ready to proceed to Polymarket integration!")